In [146]:
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_heisenberg_local_term, reduce_list, X, I2, get_perms
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector
from scipy.linalg import expm

Lx, Ly = (4, 4)
L = Lx*Ly
t = 0.125
# construct Hamiltonian
latt = qib.lattice.TriangularLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J = (1, 1, 1)
h = (3, -1, 1)
hamil = qib.HeisenbergHamiltonian(field, J, h).as_matrix()
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=10)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

hloc = construct_heisenberg_local_term(J, h, ndim=3)
#hloc2 = construct_heisenberg_local_term((0, J[1], 0), (0,0,0))
#perms_1 = [[0, 1, 3, 4, 6, 7], [1, 2, 4, 5, 7, 8], [2, 0, 5, 3, 8, 6]]
#perms_2 = [[0, 4, 1, 5, 2, 3], [4, 8, 3, 7, 5, 6], [8, 0, 6, 1, 7, 2]]
#perms_3 = [[0, 3, 1, 4, 2, 5], [3, 6, 4, 7, 5, 8], [7, 1, 6, 0, 8, 2]]
perms_1 = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12]]
perms_2 = [[0, 5, 10, 15, 3, 4, 9, 14, 2, 7, 8, 13, 1, 6, 11, 12], [5, 10, 15, 0, 4, 9, 14, 3, 7, 8, 13, 2, 6, 11, 12, 1]]
perms_3 = [[0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15], [4, 8, 12, 0, 5, 9, 13, 1, 6, 10, 14, 2, 7, 11, 15, 3]]

#Vlist_3 = [expm(-1j*t*hloc1/2) , expm(-1j*t*hloc2), expm(-1j*t*hloc1/2)]
#Vlist_1 = [expm(-1j*t*hloc1/4), expm(-1j*t*hloc2/2), expm(-1j*t*hloc1/4)]

#Vlist_start    = [expm(-1j*t*hloc/2)]*2 + [expm(-1j*t*hloc)] + [expm(-1j*t*hloc/2)]*2
#perms_extended = [perms_1] + [perms_2] + [perms_3] + [perms_2] + [perms_1]


hloc1 = construct_heisenberg_local_term((J[0], 0, 0), (h[0],0,0), ndim=3)
hloc2 = construct_heisenberg_local_term((0, J[1], 0), (0,h[1],0), ndim=3)
hloc3 = construct_heisenberg_local_term((0, 0, J[2]), (0,0,J[2]), ndim=3)
V1_5 = scipy.linalg.expm(-1j*t*hloc1/2)
V2_5 = scipy.linalg.expm(-1j*t*hloc2/2)
V3_1 = scipy.linalg.expm(-1j*t*hloc3)

Vlist_start  = [V1_5]*3+[V2_5]*3+[V3_1]*3+[V2_5]*3+[V1_5]*3
perms_extended = ([perms_1] + [perms_2] + [perms_3])*10
state = np.array(random_statevector(2**L).data)
print("Trotter error: ", 1-state_fidelity(ansatz_sparse(Vlist_start, L, perms_extended, state), expm_multiply(
    -1j * t * hamil, state)))

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Trotter error:  0.020917759897671195


In [147]:
t = 0.125

hloc1 = construct_heisenberg_local_term((0,    0, J[2]), (h[0],0,0), ndim=3)
hloc2 = construct_heisenberg_local_term((0, J[1],    0), (0,0,h[2]), ndim=3)
V1 = scipy.linalg.expm(-1j*t*hloc1)
V2 = scipy.linalg.expm(-1j*t*hloc2)

X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
I2 = np.array([[1, 0], [0, 1]])
YZ = np.kron(Y, Z)
XY = np.kron(X, Y)

Vlist_start = [YZ, V1, YZ]*3 + [XY, V2, XY]*3 
Vlist_reduced = [V1]*3 + [V2]*3 

perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +  [perms_2] + [[perms_2[0]], [perms_3[0]]] +  [perms_3] + [[perms_3[0]]]
perms_extended = perms_extended + perms_extended 
perms_ext_reduced = [perms_1] + [perms_2] + [perms_3]
perms_ext_reduced = perms_ext_reduced + perms_ext_reduced
control_layers = [0, 2, 3, 5, 6, 8, 9, 11, 12, 14, 15, 17]

state = random_statevector(2**L).data
print("Trotter error of the starting point: ", (np.linalg.norm(ansatz_sparse(Vlist_start, L, perms_extended, state) - expm_multiply(
    1j * t * hamil, state), ord=2) + np.linalg.norm(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state) - expm_multiply(
    -1j * t * hamil, state), ord=2))/2)

print('fidelity: ', (state_fidelity(ansatz_sparse(Vlist_start, L, perms_extended, state), expm_multiply(
    1j * t * hamil, state)) + state_fidelity(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state), expm_multiply(
    -1j * t * hamil, state)))/2)


Trotter error of the starting point:  0.9478262632565463
fidelity:  0.30594401712640595


In [148]:
with h5py.File(f"./results/triangularXY_ccU_SPARSE_J111_h3-11_Lx4Ly4_t0.125_layers27_niter5_rS1_2hloc.hdf5", "r") as f:
    Vlist =  f["Vlist"][:]


perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +\
      [perms_2] + [[perms_2[0]], [perms_3[0]]] +  [perms_3] + [[perms_3[0]]]
perms_extended = perms_extended*3
perms_ext_reduced = [perms_1] + [perms_2] + [perms_3]
perms_ext_reduced = perms_ext_reduced*3
non_control_layers = [i for i in range(1, 27, 3)]
control_layers = []
for i in range(27):
    if i not in non_control_layers:
        control_layers.append(i)

In [5]:
with h5py.File(f"./results/triangularXY_ccU_SPARSE_J111_h3-11_Lx4Ly4_t0.125_layers45_niter5_rS1_2hloc.hdf5", "r") as f:
    Vlist =  f["Vlist"][:]

perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +\
      [perms_2] + [[perms_2[0]], [perms_3[0]]] +  [perms_3] + [[perms_3[0]]]
perms_extended = perms_extended*5
perms_ext_reduced = [perms_1] + [perms_2] + [perms_3]
perms_ext_reduced = perms_ext_reduced*5
non_control_layers = [i for i in range(1, 45, 3)]
control_layers = []
for i in range(45):
    if i not in non_control_layers:
        control_layers.append(i)

In [149]:
"""
    Test operations.
"""
#control_layers = [0, 2, 3, 5, 6, 8, 9, 11, 12, 14, 15, 17]
Vlist_reduced = []
for i, V in enumerate(Vlist):
    if i not in control_layers:
        Vlist_reduced.append(V)

state = random_statevector(2**L).data
print("Trotter error of the optimized point: ", (np.linalg.norm(ansatz_sparse(Vlist, L, perms_extended, state) - expm_multiply(
    1j * t * hamil, state), ord=2) + np.linalg.norm(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state) - expm_multiply(
    -1j * t * hamil, state), ord=2))/2)

print('infidelity: ', 1-(state_fidelity(ansatz_sparse(Vlist, L, perms_extended, state), expm_multiply(
    1j * t * hamil, state)) + state_fidelity(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state), expm_multiply(
    -1j * t * hamil, state)))/2)

Trotter error of the optimized point:  0.1869008655564678
infidelity:  0.03484348808621218


In [58]:
sys.path.append("../../src/controlled_unitary_optimizer")
sys.path.append("../../src/brickwall_ansatz")
from optimize_3q import optimize_3q 
from utils_3q import make_controlled, random_unitary

Xlists_opt = {}
#perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
perms_qc = [[0, 1], [0, 2]]
#perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2]]

for i in control_layers:
    cU = make_controlled(Vlist[i])
    f_best, err_best, Glist_best = (0, 2, None)
    for _ in range(10):
        Xlist_start = [random_unitary(4) for i in range(len(perms_qc))]
        Xlist, f_iter, err_iter = optimize_3q(L, cU, Xlist_start, perms_qc, niter=1000)
        if err_iter[-1] < err_best:
            f_best, err_best, Xlist_best = (f_iter[-1], err_iter[-1], Xlist)
    print("Best f: ", f_best)
    print("Best err: ", err_best)
    Xlists_opt[i] = Xlist_best

Best f:  -7.999995816099407
Best err:  0.001060746521535991
Best f:  -7.999978909719291
Best err:  0.0030376591171728928
Best f:  -7.999998206358379
Best err:  0.0011247258803116166
Best f:  -7.999996307784924
Best err:  0.0013381765324806756
Best f:  -7.999999105232634
Best err:  0.0005016491844535304
Best f:  -7.9999867106412665
Best err:  0.0024163690395603336
Best f:  -7.999988561175246
Best err:  0.0018211263226523796
Best f:  -7.9999989488555485
Best err:  0.0005130924719781017
Best f:  -7.999999981588875
Best err:  6.878506925748454e-05
Best f:  -7.999995391117334
Best err:  0.0019000402082698818
Best f:  -7.999929139528636
Best err:  0.004231186697013122
Best f:  -7.999988078783838
Best err:  0.0017371323936670082
Best f:  -7.999982301372274
Best err:  0.0021318433031742136
Best f:  -7.999954348948856
Best err:  0.004779481621560131
Best f:  -7.999954018548193
Best err:  0.004283395610026947
Best f:  -7.999992470941968
Best err:  0.0018460518563181776
Best f:  -7.99998997416894

In [51]:
# Save the optimal gates.

for i in control_layers:
    with h5py.File(f"./results/triangularXY_ccU_SPARSE_J111_h3-11_Lx4Ly4_t0.125_layers45_niter5_rS1_2hloc_DECOMPOSE_n{len(perms_qc)}_layer{i}.hdf5", "w") as file:
        file.create_dataset(f"Xlist_{i}", data=Xlists_opt[i])
        

In [150]:
perms_qc = [[0, 1], [0, 2]]
#Xlists_opt = {}
for i in control_layers:
    with h5py.File(f"./results/triangularXY_ccU_SPARSE_J111_h3-11_Lx4Ly4_t0.125_layers45_niter5_rS1_2hloc_DECOMPOSE_n{len(perms_qc)}_layer{i}.hdf5", "r") as file:
        Xlists_opt[i] = file[f"Xlist_{i}"][:]
        

In [164]:
"""
    Now here is to compare the performance of the ccU circuit
    with the 1st and 2nd order Trotter circuits, in terms of 
    gate count vs Trotter error. I demonstrate it on L=10 system.
"""

from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp


def controlled_trotter(c1, L, Lx, Ly, J, h, perms_1, perms_2, perms3, dag=False, dt=t/2):
    t = c1/2
    nsteps = int(t/dt)
    t = t/nsteps
    
    #indices = oc.SplittingMethod.suzuki(3, 1).indices
    #coeffs = oc.SplittingMethod.suzuki(3, 1).coeffs
    indices = [0, 1, 2]
    coeffs = [1, 1, 1]
    perms_ext = [perms_1, perms_2, perms_3]*len(indices)
    
    hloc1 = construct_heisenberg_local_term((J[0], 0   ,    0), (0, h[1],    0), ndim=3)
    hloc2 = construct_heisenberg_local_term((0   ,    J[1], 0), (0, 0, h[2]   ), ndim=3)
    hloc3 = construct_heisenberg_local_term((0   , 0   , J[2]), (h[0], 0,    0), ndim=3)
    hlocs = (hloc1, hloc2, hloc3)
    cgates = ((CXGate, CZGate), 
              (CXGate, CYGate), 
              (CZGate, CYGate))

    K = []
    for i, perms in enumerate(perms_ext):
        sub = int(i//3)
        index = indices[sub]
        perm = perms[0]
        K_layer = [None for _ in range(L)]
        for j in range(len(perm)//2):
            K_layer[perm[2*j]] =  cgates[index][0]
            K_layer[perm[2*j+1]] =  cgates[index][1]
        K.append(K_layer)

    Vlist_start = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hlocs[i]))
    Vlist_gates = []
    for V in Vlist_start:
        qc2 = qiskit.QuantumCircuit(2)
        qc2.unitary(V, [0, 1], label='str')
        Vlist_gates.append(qc2)
    
    qc = qiskit.QuantumCircuit(L+1)
    for n in range(nsteps):
        for layer, qc_gate in enumerate(Vlist_gates):

            perms = perms_1
            qc.x(L)
            for j in range(L):
                if K[3*layer][j]:
                    qc.append(K[3*layer][j](), [L, L-1-j])
            qc.x(L)
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])
            qc.x(L)
            for j in range(L):
                if K[3*layer][j]:
                    qc.append(K[3*layer][j](), [L, L-1-j])
            qc.x(L)

            perms = perms_2
            qc.x(L)
            for j in range(L):
                if K[3*layer+1][j]:
                    qc.append(K[3*layer+1][j](), [L, L-1-j])
            qc.x(L)
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])

            qc.x(L)
            for j in range(L):
                if K[3*layer+1][j]:
                    qc.append(K[3*layer+1][j](), [L, L-1-j])
            qc.x(L)

            perms = perms_3
            qc.x(L)
            for j in range(L):
                if K[3*layer+2][j]:
                    qc.append(K[3*layer+2][j](), [L, L-1-j])
            qc.x(L)
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])
            qc.x(L)
            for j in range(L):
                if K[3*layer+2][j]:
                    qc.append(K[3*layer+2][j](), [L, L-1-j])
            qc.x(L)
    return qc

trotter1_cxs_01 = []
trotter1_errs_01 = []
for t_ in [t, 2*t, 4*t]:
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+1)
    qc_ext1.initialize(state, [i for i in range(L)])
    qc_ext1.append(controlled_trotter(2*t_, L, Lx, Ly, J, h, perms_1, perms_2, perms_3).to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv1_T = execute(transpile(qc_ext1), backend).result().get_statevector().data
    
    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    qc_ext2.append(controlled_trotter(2*t_, L, Lx, Ly, J, h, perms_1, perms_2, perms_3).to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2_T = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    exact_v1 = np.kron(ket_0, expm_multiply(1j * t_ * hamil, state))
    exact_v2 = np.kron(ket_1, expm_multiply(-1j * t_ * hamil, state))
    
    #err = (np.linalg.norm(sv1_T-exact_v1, ord=2) + np.linalg.norm(sv2_T-exact_v2, ord=2))/2
    err = 1-(state_fidelity(sv1_T, exact_v1) + state_fidelity(sv2_T, exact_v2))/2

    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, optimization_level=2, basis_gates=noise_model.basis_gates+['initialize', 'unitary', 'cx', 'u3']))
    count_ops = dag.count_ops()

    trotter1_cxs_01.append(count_ops['cx']+count_ops['unitary'])
    trotter1_errs_01.append(err)

    print(f"t={t}, Gate count: ", count_ops, " Norm error: ", err)

t=0.125, Gate count:  {'initialize': 1, 'u3': 178, 'cx': 248, 'unitary': 288, 'rz': 124}  Norm error:  0.11069367684130982
t=0.125, Gate count:  {'initialize': 1, 'u3': 338, 'cx': 488, 'unitary': 576, 'rz': 244}  Norm error:  0.21758798490161269
t=0.125, Gate count:  {'initialize': 1, 'u3': 658, 'cx': 968, 'unitary': 1152, 'rz': 484}  Norm error:  0.32443982086471923


In [163]:
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp

m = [[0, 2, 3, 5, 6, 8], [9, 11, 12, 14, 15, 17], [18, 20, 21, 23, 24, 26]]
cs = [(CXGate, CZGate), (CXGate, CYGate), (CZGate, CYGate)]
m_ = {}
for i, l in enumerate(m):
    for ind in l:
        m_[ind] = cs[i]

qc = qiskit.QuantumCircuit(L+1)
qc.x(L)
for i, V in enumerate(Vlist):
    layer = i
    if i in control_layers:
        Glist = Xlists_opt[i]
        qc_3 = qiskit.QuantumCircuit(3)
        for j, G in enumerate(Glist):
            qc_3.unitary( G, (3-1-perms_qc[j][1], 3-1-perms_qc[j][0]))

        for perm in perms_extended[layer]:
            for j in range(L//2):
                qc.append(qc_3.to_gate(), [L-perm[2*j]-1, L-perm[2*j+1]-1, L])

        """for perm in perms_extended[layer]:
            for j in range(L//2):
                qc.append(m_[i][0](),  [L, L-1-perm[2*j]])
                qc.append(m_[i][1](),  [L, L-1-perm[2*j+1]])"""
    
    else:
        for perm in perms_extended[layer]:
            for j in range(L//2):
                qc.unitary(V, [L-perm[2*j]-1, L-perm[2*j+1]-1])
qc.x(L)

ccU_errs_0125 = []
ccU_cxs_0125 = []
for t_ in [t, 2*t, 4*t]:
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+1)
    qc_ext1.initialize(state, [i for i in range(L)])
    for i in range(int(t_/t)):
        qc_ext1.append(qc.to_gate(), [i for i in range(L+1)])

    
    backend = Aer.get_backend("statevector_simulator")
    sv1 = execute(transpile(qc_ext1), backend).result().get_statevector().data

    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    for i in range(int(t_/t)):
        qc_ext2.append(qc.to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2 = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    exact_v1 = np.kron(ket_0, expm_multiply(1j * t_ * hamil, state))
    exact_v2 = np.kron(ket_1, expm_multiply(-1j * t_ * hamil, state))
    #err = (np.linalg.norm(sv1-exact_v1, ord=2) + np.linalg.norm(sv2-exact_v2, ord=2))/2
    err = 1-(np.abs(np.vdot(sv1, exact_v1))**2+np.abs(np.vdot(sv2, exact_v2))**2)/2
    
    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, optimization_level=2, basis_gates=['cx', 'initialize', 'u3', 'unitary']))
    count_ops = dag.count_ops()
    #print(count_ops)
    
    ccU_errs_0125.append(err)
    ccU_cxs_0125.append(count_ops['unitary'])
    print(f"t={t}, Gate Count: ", count_ops, " State infidelity: ", err)
    #print("Fidelity: ", 1-err)

t=0.125, Gate Count:  {'initialize': 1, 'u3': 2, 'unitary': 432}  State infidelity:  0.055650091771546895
t=0.125, Gate Count:  {'initialize': 1, 'u3': 3, 'unitary': 864}  State infidelity:  0.1274828128881429
t=0.125, Gate Count:  {'initialize': 1, 'u3': 5, 'unitary': 1728}  State infidelity:  0.2541465999898136


In [ ]:
# 27 layers
# for perms_qc length 5, you can compare it to Trotter 1st order r=3
# for perms_qc length 4, you can compare it to Trotter 2nd order with r=1
# for perms_qc length 3, you can compare it to Trotter 1st order with r=2 & r=3


# 45 layers
# Trotter 2nd with r = 2.

In [165]:
print(trotter1_cxs_01)
print(trotter1_errs_01)

[536, 1064, 2120]
[0.11069367684130982, 0.21758798490161269, 0.32443982086471923]


In [166]:
print(trotter2_cxs_01)
print(trotter2_errs_01)

[448, 880, 1744]
[0.10817339251744418, 0.2016679121793158, 0.3595905732053003]


In [167]:
print(ccU_errs_0125)
print(ccU_cxs_0125)

[0.055650091771546895, 0.1274828128881429, 0.2541465999898136]
[432, 864, 1728]


In [ ]:
trHeis_trotter1_cxs_01 = [536, 1064, 2120]
trHeis_trotter1_errs_01=[0.11069367684130982, 0.21758798490161269, 0.32443982086471923]
trHeis_trotter2_cxs_01=[448, 880, 1744]
trHeis_trotter2_errs_01=[0.10817339251744418, 0.2016679121793158, 0.3595905732053003]
trHeis_ccU_errs_0125=[0.055650091771546895, 0.1274828128881429, 0.2541465999898136]
trHeis_ccU_cxs_0125=[432, 864, 1728]